<img src='https://gitlab.eumetsat.int/eumetlab/oceans/ocean-training/tools/frameworks/-/raw/main/img/OSI-SAF_banner.png' align='right' width='100%'/>

<a href="../Index.ipynb" target="_blank"><< Index</a>
<br>
<a href="./1_1a_OSI_SAF_SST_data_access_IFREMER_Opensearch.ipynb" target="_blank"><< Accessing Metop SST metagranule data through IFREMER opensearch queries</a>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<a href="./1_1c_OSI_SAF_SST_data_access_Eumetsat_data_store.ipynb" target="_blank">Accessing data through Eumetsat data store >></a>

<font color="#138D75">**EUMETSAT OSI SAF Training Service**</font> <br>
**Copyright:** 2024 EUMETSAT <br>
**License:** MIT

<html>
  <div style="width:100%">
    <div style="float:left"><a href="https://mybinder.org/v2/git/https%3A%2F%2Fgitlab.eumetsat.int%2Feumetlab%2Foceans%2Focean-training%2Fsensors%2Flearn-osi-saf-sst/HEAD?labpath=1_OSI_SAF_sst_introductory%2F1_1b_OSI_SAF_SST_data_access_IFREMER_FTP.ipynb"><img src="https://mybinder.org/badge_logo.svg" alt="Open in Binder"></a></div>
    <div style="float:left"><p>&emsp;</p></div>
  </div>
</html>

<div class="alert alert-block alert-success">
<h3>Learn OSI SAF sea surface temperature: Introductory</h3></div>

<div class="alert alert-block alert-warning">
    
<b>PREREQUISITES </b>
    
This notebook has the following prerequisites:
- **<a href="https://osi-saf.eumetsat.int/register" target="_blank">A EUMETSAT OSI SAF account</a>** if you are using or plan to use the EUMETSAT OSI SAF data.

There are no prerequisite notebooks for this module.
</div>
<hr>

# 1.1b Accessing OSI SAF SST products through the IFREMER FTP server
### Data used 

| Dataset | EUMETSAT collection ID | OSI SAF website description | OSI SAF identifier |
|:-----------------:|:-----------------:|:-----------------:|:-----------------:|
| Global Metop Sea Surface Temperature | EO:EUM:DAT:METOP:GLB-SST-NC | <a href="https://osi-saf.eumetsat.int/products/osi-201-b" target="_blank">Description</a>| OSI-201-b |
| North Atlantic Regional Sea Surface Temperature | EO:EUM:DAT:MULT:OSSTNARN | <a href="https://osi-saf.eumetsat.int/products/osi-202-c" target="_blank">Description</a>| OSI-202-c |
| Meteosat Sea Surface Temperature | EO:EUM:DAT:METEOSAT:OSIHSST-NC | <a href="https://osi-saf.eumetsat.int/products/osi-206-a" target="_blank">Description</a>| OSI-206-a | 
| GOES-East Sea Surface Temperature | EO:EUM:DAT:GOES:OSIHSST-NCC | <a href="https://osi-saf.eumetsat.int/products/osi-207-b" target="_blank">Description</a>| OSI-207-b |
| MSG/SEVIRI Sea Surface Temperature data record | EO:EUM:DAT:MSG:OSI-250 | <a href="https://osi-saf.eumetsat.int/products/osi-250" target="_blank">Description</a>| OSI-250 | 
| Meteosat Indian Ocean Sea Surface Temperature | - | <a href="https://osi-saf.eumetsat.int/products/osi-io-sst" target="_blank">Description</a>| OSI-IO-sst |


### Learning outcomes

At the end of this notebook you will know;
* How to download different OSI SAF sea surface temperature (SST) products using FTP access

### Outline

Data from OSI SAF is available through multiple sources. Here we will guide you through FTP access to OSI SAF SST Data.

<div class="alert alert-info" role="alert">

## <a id='TOC-TOP'></a>Contents

</div>
    
 1. [Creating our workspace](#section1)
 1. [Setting login information](#section2)
 1. [Downloading via FTP requests](#section3)

<hr>

<div class="alert alert-info" role="alert">

## <a id='section1'></a>1. Creating the workspace
[Back to top](#TOC-TOP)

</div>

We begin by importing all of the libraries that we need to run this notebook. If you have built your python using the environment file provided in this repository, then you should have everything you need. For more information on building environment, please see the repository **<a href="../README.md" target="_blank">README</a>**.

In [1]:
# library imports 
import os           # a library that allows to access to basic operating system commands like making directories
import json         # a library that helps with JSON format files
import ftplib       # a library that supports FTP protocol use
import warnings     # a library that controls python warnings
import datetime     # a library that controls dates and hours
warnings.filterwarnings('ignore')

Next we will create a download directory to store the products we will download in this notebook.

In [2]:
download_dir = os.path.join(os.getcwd(), "products")
os.makedirs(download_dir, exist_ok=True)

<div class="alert alert-info" role="alert">

## <a id='section2'></a>2. Setting login information
[Back to top](#TOC-TOP)

</div>

We will access OSI SAF data from the OSI SAF Lower-Mid-Latitude Centre FTP server in the same way as in the <a href="./1_1a_OSI_SAF_SST_data_access_IFREMER_Opensearch.ipynb" target="_blank">previous chapter</a>.

#### Option 1: creating  `.eumetsat_osi_saf_sst_credentials` in our home directory.

If you created the `.eumetsat_osi_saf_sst_credentials` in the <a href="./1_1a_OSI_SAF_SST_data_access_IFREMER_Opensearch.ipynb" target="_blank">previous chapter</a>, you just need to execute the command in the following cell : the credentials to access the FTP server will be load from the file you previously created.

Reading credential information

In [3]:
# read credentials
with open(os.path.join(os.path.expanduser("~"),'.eumetsat_osi_saf_sst_credentials')) as json_file:
  credentials = json.load(json_file)

Loading credentials

In [4]:
username = credentials['username'] 
password = credentials['password']

#### Option 2: provide credentials directly
You can provide your credentials directly as follows; by uncommenting the following lines.

In [5]:
# username = "anonymous"
# password = "anonymous"

You must replace <provided_username> and <provided_password> with the information provided to you after you have registered to the OSI SAF website and mentionned your interest in SST data.

Once you have created an EUMETSAT OSI SAF account, the generic password and credentials to access OSI SAF Lower-Mid-Latitude  server should have been sent to you via email.

Note: this method is convenient in the short term, but is not really recommended as you have to put your provided username and password in this notebook, and run the risk of accidentally sharing them. This method also requires you to authenticate on a notebook-by-notebook basis.

<div class="alert alert-info" role="alert">

## <a id='section3'></a>3. Launch FTP requests
[Back to top](#TOC-TOP)

</div>

In this section, we will download different SST products from OSI-SAF. Each will be used in a different mapping section later on.

### Global Metop Sea Surface Temperature :
#### Satellite Metop B
Let's start by creating a specific folder to hold the Global Metop Sea Surface products for satellite metop B

In [6]:
download_dir = os.path.join(os.getcwd(), "products", "global_GHRSST")
os.makedirs(download_dir, exist_ok=True)

Set the download path

In [7]:
# Select date
Year, Month, Day = 2023, 7, 15

In [8]:
day_of_year = "{:03d}".format(datetime.datetime(Year, Month, Day).timetuple().tm_yday) # Calcul the day number of the year
Month_str = "{:02d}".format(Month)
Day_str = "{:02d}".format(Day)

path = '/ifremer/cersat/projects/osisaf/sst/l3c/global/glob_avhrr_metop_b/' + str(Year) + '/' + day_of_year + '/'
filename = str(Year) + Month_str + Day_str + \
        '000000-OSISAF-L3C_GHRSST-SSTsubskin-AVHRR_SST_METOP_B_GLB-sstglb_metop01_'+ \
        str(Year) + Month_str + Day_str + '_000000-v02.0-fv01.0.nc'

In [9]:
# connect ftp and download file
ftp = ftplib.FTP("ftp.ifremer.fr") 
ftp.login(username, password)
ftp.cwd(path)
ftp.retrbinary("RETR " + filename, open(os.path.join(os.getcwd(), 'products', 'global_GHRSST', filename), 'wb').write)
ftp.quit()

'221 Goodbye.'

###  North Atlantic Regional Sea Surface Temperature:

Let's start by creating a specific folder to hold the NAR Sea Surface Temperature products.

In [10]:
download_dir = os.path.join(os.getcwd(), "products", "NAR")
os.makedirs(download_dir, exist_ok=True)

NAR SST products are accessible from diverse type of satellite. We will use the examples **Noaa20** and **Metop-B**. The other data can be access by the same way as show on the following instruction.

#### Noaa 20 data
Let's start by creating a specific folder to hold the data from Noaa 20.

In [11]:
download_dir = os.path.join(os.getcwd(), "products", "NAR", "Noaa20")
os.makedirs(download_dir, exist_ok=True)

We set the download path where the datas are and we will chose a file in this path.

In [12]:
# Select date
Year, Month, Day = 2023, 8, 12

In [13]:
day_of_year = "{:03d}".format(datetime.datetime(Year, Month, Day).timetuple().tm_yday) # Calcul the day number of the year
Month_str = "{:02d}".format(Month)
Day_str = "{:02d}".format(Day)

path = '/ifremer/cersat/projects/osisaf/sst/l3c/north_atlantic/nar_viirs_noaa_20/' + str(Year) + '/' + str(day_of_year)
filename = str(Year) + Month_str + Day_str + \
        '010000-OSISAF-L3C_GHRSST-SSTsubskin-VIIRS_SST_NOAA20_NAR-sstnar_noaa20_' + \
        str(Year) + Month_str + Day_str + '_010000-v02.0-fv01.0.nc'

In [14]:
# connect ftp and download file
ftp = ftplib.FTP("ftp.ifremer.fr") 
ftp.login(username, password)
ftp.cwd(path)
ftp.retrbinary("RETR " + filename, open(os.path.join(os.getcwd(), 'products', 'NAR', 'Noaa20', filename), 'wb').write)
ftp.quit()

'221 Goodbye.'

#### Metop B data
Let's start by creating a specific folder to hold the data from Metop B.

In [15]:
download_dir = os.path.join(os.getcwd(), "products", "NAR", "MetopB")
os.makedirs(download_dir, exist_ok=True)

We set the dowload path where the datas are and we will chose a file in this path.

In [16]:
# Select date
Year, Month, Day = 2023, 6, 8

In [17]:
day_of_year = "{:03d}".format(datetime.datetime(Year, Month, Day).timetuple().tm_yday) # Calcul the day number of the year
Month_str = "{:02d}".format(Month)
Day_str = "{:02d}".format(Day)

path = '/ifremer/cersat/projects/osisaf/sst/l3c/north_atlantic/nar_avhrr_metop_b/' + str(Year) + '/' + day_of_year
filename = str(Year) + Month_str + Day_str + \
        '100000-OSISAF-L3C_GHRSST-SSTsubskin-AVHRR_SST_METOP_B_NAR-sstnar_metop01_' + \
        str(Year) + Month_str + Day_str +'_100000-v02.0-fv01.0.nc'

In [18]:
# connect ftp and download file
ftp = ftplib.FTP("ftp.ifremer.fr") 
ftp.login(username, password)
ftp.cwd(path)
ftp.retrbinary("RETR " + filename, open(os.path.join(os.getcwd(), 'products', 'NAR', 'MetopB', filename), 'wb').write)
ftp.quit()

'221 Goodbye.'

### Meteosat Sea Surface Temperature :

Let's start by creating a specific folder to hold the Meteosat Sea Surface Temperature products

In [19]:
download_dir = os.path.join(os.getcwd(), "products", "meteosat")
os.makedirs(download_dir, exist_ok=True)

Set the download path

In [20]:
# Select date
Year, Month, Day = 2023, 6, 9

In [21]:
day_of_year = "{:03d}".format(datetime.datetime(Year, Month, Day).timetuple().tm_yday) # Calcul the day number of the year
Month_str = "{:02d}".format(Month)
Day_str = "{:02d}".format(Day)

path = '/ifremer/cersat/projects/osisaf/sst/l3c/east_atlantic_west_indian/meteosat/' + str(Year) + '/' + day_of_year
filename = str(Year) + Month_str + Day_str + \
        '000000-OSISAF-L3C_GHRSST-SSTsubskin-SEVIRI_SST-ssteqc_meteosat10_' + \
        str(Year) + Month_str + Day_str + '_000000-v02.0-fv01.0.nc'

In [22]:
# connect ftp and download file
ftp = ftplib.FTP("ftp.ifremer.fr") 
ftp.login(username, password)
ftp.cwd(path)
ftp.retrbinary("RETR " + filename, open(os.path.join(os.getcwd(), 'products', 'meteosat', filename), 'wb').write)
ftp.quit()

'221 Goodbye.'

### GOES-East Sea Surface Temperature

Let's start by creating a specific folder to hold the GOES-East Sea Surface Temperature products.

In [23]:
download_dir = os.path.join(os.getcwd(), "products", "GOES-E")
os.makedirs(download_dir, exist_ok=True)

We set the dowload path where the datas are and we will chose a file in this path.

In [24]:
# Select date
Year, Month, Day = 2023, 6, 9

In [25]:
day_of_year = "{:03d}".format(datetime.datetime(Year, Month, Day).timetuple().tm_yday) # Calcul the day number of the year
Month_str = "{:02d}".format(Month)
Day_str = "{:02d}".format(Day)

path = '/ifremer/cersat/projects/osisaf/sst/l3c/west_atlantic_east_pacific/goes/'+ str(Year) + '/' + day_of_year
filename = str(Year) + Month_str + Day_str + \
        '000000-OSISAF-L3C_GHRSST-SSTsubskin-GOES16-ssteqc_goes16_' + \
        str(Year) + Month_str + Day_str + '_000000-v02.0-fv01.0.nc'

In [26]:
# connect ftp and download file
ftp = ftplib.FTP("ftp.ifremer.fr") 
ftp.login(username, password)
ftp.cwd(path)
ftp.retrbinary("RETR " + filename, open(os.path.join(os.getcwd(), 'products', 'GOES-E', filename), 'wb').write)
ftp.quit()

'221 Goodbye.'

### MSG/SEVIRI Sea Surface Temperature data record

Let's start by creating a specific folder to hold the MSG/SEVERI Sea Surface Temperature products.\
These products are available until 2012-12-31.

In [27]:
download_dir = os.path.join(os.getcwd(), "products", "MSG_record")
os.makedirs(download_dir, exist_ok=True)

We set the dowload path where the datas are and we will chose a file in this path.

In [28]:
# Select date
Year, Month, Day = 2012, 12, 31

In [29]:
Month_str = "{:02d}".format(Month)
Day_str = "{:02d}".format(Day)

path = '/ifremer/cersat/projects/osisaf/sst/l3c/east_atlantic_west_indian/meteosat_data_record/v1/' + str(Year) + '/' + Month_str + '/' + Day_str
filename = str(Year) + Month_str + Day_str + \
        '000000-OSISAF-L3C_GHRSST-SSTsubskin-SEVIRI_SST-dr_ssteqc_meteosat09_' + \
        str(Year) + Month_str + Day_str + '_000000-v02.0-fv01.0.nc'

In [30]:
# connect ftp and download file
ftp = ftplib.FTP("ftp.ifremer.fr") 
ftp.login(username, password)
ftp.cwd(path)
ftp.retrbinary("RETR " + filename, open(os.path.join(os.getcwd(), 'products', 'MSG_record', filename), 'wb').write)
ftp.quit()

'221 Goodbye.'

### Meteosat Indian Ocean Sea Surface Temperature

Let's start by creating a specific folder to hold the Meteosat Indian Ocean Sea Surface Temperature.

In [31]:
download_dir = os.path.join(os.getcwd(), "products", "MSGIO")
os.makedirs(download_dir, exist_ok=True)

We set the download path where the datas are and we will chose a file in this path.

In [32]:
# Select date
Year, Month, Day = 2023, 6, 9

In [33]:
day_of_year = "{:03d}".format(datetime.datetime(Year, Month, Day).timetuple().tm_yday) # Calcul the day number of the year
Month_str = "{:02d}".format(Month)
Day_str = "{:02d}".format(Day)

path = '/ifremer/cersat/projects/osisaf/sst/l3c/indian/meteosat/' + str(Year) + '/' + day_of_year
filename = str(Year) + Month_str + Day_str + \
        '000000-OSISAF-L3C_GHRSST-SSTsubskin-SEVIRI_IO_SST-ssteqc_meteosat09_' + \
        str(Year) + Month_str + Day_str + '_000000-v02.0-fv01.0.nc'

In [34]:
# connect ftp and download file
ftp = ftplib.FTP("ftp.ifremer.fr") 
ftp.login(username, password)
ftp.cwd(path)
ftp.retrbinary("RETR " + filename, open(os.path.join(os.getcwd(), 'products', 'MSGIO', filename), 'wb').write)
ftp.quit()

'221 Goodbye.'

### IASI Metop B Sea Surface Temperature

Let's start by creating a specific folder to hold the IASI Metop B Sea Surface Temperature.

In [35]:
download_dir = os.path.join(os.getcwd(), "products", "iasi")
os.makedirs(download_dir, exist_ok=True)

In [36]:
# Select date
Year, Month, Day = 2024, 1, 9

In [37]:
day_of_year = "{:03d}".format(datetime.datetime(Year, Month, Day).timetuple().tm_yday) # Calcul the day number of the year
Month_str = "{:02d}".format(Month)
Day_str = "{:02d}".format(Day)

path = '/ifremer/cersat/projects/osisaf/sst/l2p/global/iasi_metop_b/' + str(Year) + '/' + day_of_year

In [38]:
# connect ftp and download file
ftp = ftplib.FTP("ftp.ifremer.fr") 
ftp.login(username, password)
ftp.cwd(path)
file = ftp.nlst()
for filename in file:
    ftp.retrbinary("RETR " + filename, open(os.path.join(os.getcwd(), 'products', 'iasi', filename), 'wb').write)
ftp.quit()

'221 Goodbye.'

<hr>
<a href="../Index.ipynb" target="_blank"><< Index</a>
<br>
<a href="./1_1a_OSI_SAF_SST_data_access_IFREMER_Opensearch.ipynb" target="_blank"><< Accessing Metop SST metagranule data through IFREMER opensearch queries</a>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<a href="./1_1c_OSI_SAF_SST_data_access_Eumetsat_data_store.ipynb" target="_blank">Accessing data through Eumetsat data store >></a>
<hr>
<a href="https://gitlab.eumetsat.int/eumetlab/ocean">View on GitLab</a> | <a href="https://training.eumetsat.int/">EUMETSAT Training</a> | <a href=mailto:ops@eumetsat.int>Contact helpdesk for support </a> | <a href=mailto:training@eumetsat.int>Contact our training team to collaborate on and reuse this material</a></span></p>